# Test 2 - Data preparation

The following work illustrates preparing well log data so it can be ready to be analysed, including cleaning bad data, TVD calculation, seafloor extrapolation, and well log normalization. After that, coal determination can be conducted due to clear distinction on well log.

## Task list
* Pick well to analyze based on availability data
* Cleaning data
* Log normalization
* TVD calculation
* Seafloor extrapolation
* Predict missing density log (?)


In [1]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt

In [2]:
data = pd.read_csv("FORCE 2020 lithofacies prediction from well logs competition/CSV_train.csv", sep=';')
data.rename(columns = {'Z_LOC':'DEPTH'}, inplace = True)
data.describe()

,DEPTH_MD,X_LOC,Y_LOC,DEPTH,CALI,RSHA,RMED,RDEP,RHOB,GR,...,ROP,DTS,DCAL,DRHO,MUDWEIGHT,RMIC,ROPA,RXO,FORCE_2020_LITHOFACIES_LITHOLOGY,FORCE_2020_LITHOFACIES_CONFIDENCE
count,1.170511e+06,1.159736e+06,1.159736e+06,1.159736e+06,1.082634e+06,630650.000000,1.131518e+06,1.159496e+06,1.009242e+06,1.170511e+06,...,535071.000000,174613.000000,298833.000000,987857.000000,316151.000000,176160.000000,192325.000000,327427.000000,1.170511e+06,1.170332e+06
mean,2.184087e+03,4.856310e+05,6.681276e+06,-2.138527e+03,1.318568e+01,10.694664,4.986978e+00,1.069103e+01,2.284987e+00,7.091370e+01,...,137.367965,204.655019,1.223849,0.012196,1.216329,7.796809,23.505069,-95.779496,6.138598e+04,1.164258e+00
std,9.971821e+02,3.455641e+04,1.281524e+05,9.709426e+02,3.798907e+00,100.642597,5.467269e+01,1.139480e+02,2.532835e-01,3.423149e+01,...,1539.383558,71.068461,54.372859,7.477798,10.180834,89.741807,22.632800,348.980030,1.389170e+04,4.451180e-01
min,1.360860e+02,4.268988e+05,6.406641e+06,-5.395563e+03,2.344000e+00,0.000100,-8.418695e-03,3.170056e-02,7.209712e-01,1.092843e-01,...,-0.117977,69.163177,-12.215459,-7429.338867,0.125818,0.056586,-999.250122,-999.900024,3.000000e+04,1.000000e+00
25%,1.418597e+03,4.547996e+05,6.591327e+06,-2.811502e+03,9.429712e+00,0.854120,9.140862e-01,9.102396e-01,2.092203e+00,4.762722e+01,...,5.628000,155.936707,0.148438,-0.009253,0.143792,0.891272,11.281895,0.687681,6.500000e+04,1.000000e+00
50%,2.076605e+03,4.769203e+05,6.737311e+06,-2.042785e+03,1.255575e+01,1.399020,1.443584e+00,1.439000e+00,2.321228e+00,6.836763e+01,...,17.799999,188.200653,0.557320,0.001752,0.155774,1.967156,20.131153,1.366672,6.500000e+04,1.000000e+00
75%,2.864393e+03,5.201532e+05,6.784886e+06,-1.391866e+03,1.671075e+01,3.099348,2.680930e+00,2.557220e+00,2.488580e+00,8.903551e+01,...,34.812794,224.645081,1.299655,0.021702,1.168307,5.084564,31.138481,3.420790,6.500000e+04,1.000000e+00
max,5.436632e+03,5.726328e+05,6.856661e+06,-1.110860e+02,2.827900e+01,2193.904541,1.988616e+03,1.999887e+03,3.457820e+00,1.076964e+03,...,47015.125000,676.578125,10011.422852,2.836938,185.730927,10000.000000,742.797852,35930.671875,9.900000e+04,3.000000e+00


In [3]:
data.columns

Index(['WELL', 'DEPTH_MD', 'X_LOC', 'Y_LOC', 'DEPTH', 'GROUP', 'FORMATION',
       'CALI', 'RSHA', 'RMED', 'RDEP', 'RHOB', 'GR', 'SGR', 'NPHI', 'PEF',
       'DTC', 'SP', 'BS', 'ROP', 'DTS', 'DCAL', 'DRHO', 'MUDWEIGHT', 'RMIC',
       'ROPA', 'RXO', 'FORCE_2020_LITHOFACIES_LITHOLOGY',
       'FORCE_2020_LITHOFACIES_CONFIDENCE'],
      dtype='object')

In [5]:
data['FORCE_2020_LITHOFACIES_LITHOLOGY'].nunique()

12

In [6]:
data.rename(columns = {'FORCE_2020_LITHOFACIES_LITHOLOGY':'LITHOLOGY'}, inplace = True)

In [7]:
well = data[['WELL','DEPTH','CALI','RSHA','RMED','RDEP','RXO','GR','NPHI','PEF','RHOB','SP','BS','DRHO','DTC','DTS', 'LITHOLOGY']]

In [8]:
well_input = well[well['WELL'].isin(['33/9-17', '35/11-11', '31/2-19 S','35/12-1', '25/2-7', '25/4-5', '15/9-15', '16/4-1'])]
well_input.to_csv('wellinputspecproject.csv')

In [1]:
well[well['WELL']=='15/9-17'].describe()

NameError: name 'well' is not defined

## Pick well

In [3]:
#well_datacount=well.groupby('WELL').agg(lambda x:x.count())
well_datacount=well.groupby(['WELL'], as_index = False)
well_datacount['DTS'].count()

,WELL,DTS
0,15/9-13,0
1,15/9-15,0
2,15/9-17,0
3,16/1-2,0
4,16/1-6 A,0
...,...,...
93,35/9-5,4942
94,35/9-6 S,0
95,36/7-3,0
96,7/1-1,0


In [94]:
#More research about groupby Dataframes

well_datacount.head() #Check data
well_datacount['WELL'].unique() #Show well names in an array
well_datacount.nunique() #Show how many unique values are in each well & each column
well_datacount.size() #Show how many row data on each well
well_datacount.get_group('7/1-1').index #Show the location data of a whole well, in this case Well 7/1-1

,WELL,DEPTH,CALI,RSHA,RMED,RDEP,RXO,GR,NPHI,PEF,RHOB,SP,BS,DRHO,DTC,DTS
0,15/9-13,494.5280,19.480835,NaN,1.611410,1.798681,NaN,80.200851,NaN,20.915468,1.884186,24.612379,NaN,-0.574928,161.131180,NaN
1,15/9-13,494.6800,19.468800,NaN,1.618070,1.795641,NaN,79.262886,NaN,19.383013,1.889794,23.895531,NaN,-0.570188,160.603470,NaN
2,15/9-13,494.8320,19.468800,NaN,1.626459,1.800733,NaN,74.821999,NaN,22.591518,1.896523,23.916357,NaN,-0.574245,160.173615,NaN
3,15/9-13,494.9840,19.459282,NaN,1.621594,1.801517,NaN,72.878922,NaN,32.191910,1.891913,23.793688,NaN,-0.586315,160.149429,NaN
4,15/9-13,495.1360,19.453100,NaN,1.602679,1.795299,NaN,71.729141,NaN,38.495632,1.880034,24.104078,NaN,-0.597914,160.128342,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1168471,7/1-2 S,2859.9924,8.728068,1.158282,1.141734,1.240302,NaN,257.011993,NaN,NaN,NaN,NaN,12.250001,NaN,NaN,NaN
1168472,7/1-2 S,2860.1444,8.741534,1.122814,1.122354,1.229349,NaN,252.575577,NaN,NaN,NaN,NaN,12.250001,NaN,NaN,NaN
1168473,7/1-2 S,2860.2964,8.739483,1.204416,1.208588,1.246152,NaN,263.705017,NaN,NaN,NaN,NaN,12.250001,NaN,NaN,NaN
1168474,7/1-2 S,2860.4484,8.739040,1.213172,1.226264,1.235385,NaN,272.152985,NaN,NaN,NaN,NaN,12.250001,NaN,NaN,NaN


In [5]:
#Filter well that does not have DTS log
well_nodts = well_datacount.filter(lambda x:x['DTS'].count()==0).groupby(['WELL'])
print("Wells that doesn't have DTS log: ", len(well_nodts['WELL'].count()))

#Filter well that has DTS log
well_dts = well_datacount.filter(lambda x:x['DTS'].count()!=0).groupby(['WELL'])
print("Wells that have DTS log: ", len(well_dts['WELL'].count()))

Wells that doesn't have DTS log:  66
Wells that have DTS log:  32


In [6]:
well_bs = well_datacount.filter(lambda x:x['BS'].count()!=0).groupby(['WELL'])
print("Wells that have BS log: ", len(well_bs['WELL'].count()))

well_nobs = well_datacount.filter(lambda x:x['BS'].count()==0).groupby(['WELL'])
print("Wells that doesn't have BS log: ", len(well_nobs['WELL'].count()))
list(well_nobs['WELL'].unique())

well_cali = well_datacount.filter(lambda x:x['CALI'].count()!=0).groupby(['WELL'])
print("Wells that have CALI log: ", len(well_cali['WELL'].count()))

well_bscali = well_bs.filter(lambda x:x['CALI'].count()!=0).groupby(['WELL'])
print("Wells that have BS and CALI log: ", len(well_bscali['WELL'].count()))
list(well_bscali['WELL'].unique())

well_ready = well_bscali.filter(lambda x:x['DTS'].count()!=0).groupby(['WELL'])
print("Wells that can be used: ", len(well_ready['WELL'].count()))
list(well_ready['WELL'].unique())

well_potential = well_bscali.filter(lambda x:x['DTS'].count()==0).groupby(['WELL'])
print("Wells that can be used but no DTS: ", len(well_potential['WELL'].count()))
list(well_potential['WELL'].unique())

Wells that have BS log:  69
Wells that doesn't have BS log:  29
Wells that have CALI log:  97
Wells that have BS and CALI log:  68
Wells that can be used:  30
Wells that can be used but no DTS:  38


[array(['15/9-15'], dtype=object),
 array(['16/1-2'], dtype=object),
 array(['16/1-6 A'], dtype=object),
 array(['16/10-1'], dtype=object),
 array(['16/10-2'], dtype=object),
 array(['16/11-1 ST3'], dtype=object),
 array(['16/4-1'], dtype=object),
 array(['16/7-4'], dtype=object),
 array(['25/11-15'], dtype=object),
 array(['25/11-5'], dtype=object),
 array(['25/2-13 T4'], dtype=object),
 array(['25/2-14'], dtype=object),
 array(['25/2-7'], dtype=object),
 array(['25/3-1'], dtype=object),
 array(['25/4-5'], dtype=object),
 array(['25/5-4'], dtype=object),
 array(['25/6-1'], dtype=object),
 array(['25/6-2'], dtype=object),
 array(['25/7-2'], dtype=object),
 array(['25/8-7'], dtype=object),
 array(['25/9-1'], dtype=object),
 array(['26/4-1'], dtype=object),
 array(['31/2-1'], dtype=object),
 array(['31/3-1'], dtype=object),
 array(['31/3-2'], dtype=object),
 array(['31/3-3'], dtype=object),
 array(['31/6-5'], dtype=object),
 array(['31/6-8'], dtype=object),
 array(['33/5-2'], dtype=objec

Based on log availabilities on each well and its coverage (such as BS, CALI, NPHI, RHOB, DTC, and DTS), all 98 wells can be categorized into 3 types:
* Green category (19 wells) --> Have complete logs, great coverage in the well
* Yellow category (40 wells) --> Doesn't have complete logs (DTS, PEF) or poor coverage in the well (NPHI, Sonic), but can still be a backup for lithology interpretation through petrophysics, can also be used as training set
* Red category (39 wells) --> Doesn't have BS-CALI logs, poor NPHI-RHOB coverages, unable to perform petrophysical analysis, can only be used as training set

As core data is less commonly available than well logs, only 13 wells has a por-perm core data with most of them are from yellow and red category.

*More detail about picking the wells might be continued and reorganized later with well data availability plot and well table from Google Sheet.*



## Cleaning data

In [3]:
well.describe()

,DEPTH,CALI,RSHA,RMED,RDEP,RXO,GR,NPHI,PEF,RHOB,SP,BS,DRHO,DTC,DTS
count,1.170511e+06,1.082634e+06,630650.000000,1.131518e+06,1.159496e+06,327427.000000,1.170511e+06,765409.000000,671692.000000,1.009242e+06,864247.000000,682657.000000,987857.000000,1.089648e+06,174613.000000
mean,2.184087e+03,1.318568e+01,10.694664,4.986978e+00,1.069103e+01,-95.779496,7.091370e+01,0.331966,6.319719,2.284987e+00,60.033980,11.931432,0.012196,1.133581e+02,204.655019
std,9.971821e+02,3.798907e+00,100.642597,5.467269e+01,1.139480e+02,348.980030,3.423149e+01,0.130483,10.955360,2.532835e-01,76.574359,3.330242,7.477798,2.999050e+01,71.068461
min,1.360860e+02,2.344000e+00,0.000100,-8.418695e-03,3.170056e-02,-999.900024,1.092843e-01,-0.035822,0.099718,7.209712e-01,-999.000000,6.000000,-7429.338867,7.415132e+00,69.163177
25%,1.418597e+03,9.429712e+00,0.854120,9.140862e-01,9.102396e-01,0.687681,4.762722e+01,0.241365,3.414440,2.092203e+00,32.402981,8.500000,-0.009253,8.782584e+01,155.936707
50%,2.076605e+03,1.255575e+01,1.399020,1.443584e+00,1.439000e+00,1.366672,6.836763e+01,0.326800,4.313530,2.321228e+00,55.390942,12.250001,0.001752,1.095852e+02,188.200653
75%,2.864393e+03,1.671075e+01,3.099348,2.680930e+00,2.557220e+00,3.420790,8.903551e+01,0.422951,5.968794,2.488580e+00,83.393299,12.250001,0.021702,1.407745e+02,224.645081
max,5.436632e+03,2.827900e+01,2193.904541,1.988616e+03,1.999887e+03,35930.671875,1.076964e+03,0.999570,383.129974,3.457820e+00,526.547302,26.000002,2.836938,3.204789e+02,676.578125


### Resistivity log

In [4]:
#Filter RXO (should be in range 0-2000)
well.drop(well[(well['RXO'] > 2000)|(well['RXO'] < 0)].index, inplace=True)

#Filter RMED (should be in range 0-2000)
well.drop(well[(well['RMED'] < 0)].index, inplace=True)

#Filter RDEP (should be in range 0-2000)
well.drop(well[(well['RDEP'] < 0)].index, inplace=True)

C:\Users\V330\AppData\Local\Temp\ipykernel_19304\946483472.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  well.drop(well[(well['RXO'] > 2000)|(well['RXO'] < 0)].index, inplace=True)
C:\Users\V330\AppData\Local\Temp\ipykernel_19304\946483472.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  well.drop(well[(well['RMED'] < 0)].index, inplace=True)
C:\Users\V330\AppData\Local\Temp\ipykernel_19304\946483472.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

In [5]:
well.describe()

,DEPTH,CALI,RSHA,RMED,RDEP,RXO,GR,NPHI,PEF,RHOB,SP,BS,DRHO,DTC,DTS
count,1.135874e+06,1.048081e+06,625497.000000,1.096912e+06,1.124873e+06,292797.000000,1.135874e+06,757279.000000,637187.000000,974804.000000,829629.000000,666198.000000,953423.000000,1.055021e+06,174612.000000
mean,2.180040e+03,1.313926e+01,10.479709,5.042702e+00,1.089901e+01,9.728669,7.075844e+01,0.333146,6.322318,2.285095,61.607047,11.892924,0.012057,1.133007e+02,204.655516
std,9.934684e+02,3.797780e+00,98.907708,5.551177e+01,1.155919e+02,95.309516,3.449836e+01,0.130168,11.164495,0.253094,75.520441,3.328053,7.611633,2.995230e+01,71.068361
min,1.360860e+02,2.344000e+00,0.000100,3.990377e-02,3.170056e-02,0.002298,1.092843e-01,-0.035822,0.099718,0.720971,-999.000000,6.000000,-7429.338867,7.415132e+00,69.163177
25%,1.416311e+03,9.374846e+00,0.855568,9.158213e-01,9.124687e-01,0.870567,4.730949e+01,0.243197,3.401787,2.093823,33.853458,8.500000,-0.009540,8.785166e+01,155.936787
50%,2.074336e+03,1.253965e+01,1.405724,1.452153e+00,1.437460e+00,1.574212,6.813677e+01,0.328081,4.291945,2.321909,55.834679,12.250001,0.001525,1.094258e+02,188.201027
75%,2.865187e+03,1.637324e+01,3.125447,2.679512e+00,2.554810e+00,4.039199,8.858973e+01,0.423909,5.925502,2.488597,84.735519,12.250001,0.021392,1.407531e+02,224.645546
max,5.436632e+03,2.827900e+01,2193.904541,1.988616e+03,1.999887e+03,2000.000000,1.076964e+03,0.999570,383.129974,3.457820,526.547302,26.000002,2.836938,3.204789e+02,676.578125


### DRHO

In [64]:
well['DRHO'].quantile(0.005)

-0.40624813630000006

In [65]:
drhooutlier = well[(well['DRHO'] < -0.4)]
drhooutlier['WELL'].unique()
#drhooutlier[drhooutlier['WELL']=='15/9-13']

array(['15/9-13', '15/9-17', '16/10-1', '16/10-2', '16/10-3', '16/7-5',
       '25/3-1', '25/4-5', '25/5-1', '25/8-5 S', '25/9-1', '30/3-5 S',
       '30/6-5', '31/2-9', '31/3-1', '31/3-3', '31/4-5', '31/6-5',
       '33/5-2', '33/6-3 S', '33/9-17', '34/10-21', '34/10-35', '34/2-4',
       '34/7-20', '34/7-21', '34/8-1', '34/8-3', '35/11-1', '35/11-6',
       '35/11-7', '35/8-4'], dtype=object)

In [67]:
drhooutlier[drhooutlier['WELL']=='15/9-17']

,WELL,DEPTH,CALI,RSHA,RMED,RDEP,RXO,GR,NPHI,PEF,RHOB,SP,BS,DRHO,DTC,DTS
35987,15/9-17,472.429998,19.241486,NaN,1.902191,1.932372,NaN,78.812988,NaN,NaN,2.142719,1.860228,NaN,-0.530502,162.764771,NaN
35988,15/9-17,472.581998,19.198761,NaN,1.863542,1.919194,NaN,78.271599,NaN,275.688873,2.133352,2.232189,NaN,-0.543880,163.165253,NaN
35989,15/9-17,472.733998,19.175301,NaN,1.845743,1.921902,NaN,79.380615,NaN,162.921219,2.145218,1.241609,NaN,-0.547403,163.225006,NaN
35990,15/9-17,472.885998,19.178040,NaN,1.795807,1.922999,NaN,80.989418,NaN,40.905296,2.197313,0.642905,NaN,-0.547871,163.141357,NaN
35991,15/9-17,473.037998,19.171900,NaN,1.735457,1.916189,NaN,80.969376,NaN,32.873501,2.325277,1.467502,NaN,-0.549800,163.234329,NaN
35992,15/9-17,473.189998,19.162523,NaN,1.716966,1.918546,NaN,79.811470,NaN,28.391899,2.413698,1.136028,NaN,-0.584724,163.407501,NaN
35993,15/9-17,473.341998,19.156300,NaN,1.735299,1.923234,NaN,79.138283,NaN,29.591883,2.506036,0.470084,NaN,-0.592400,163.504196,NaN
35994,15/9-17,473.493998,19.174889,NaN,1.719543,1.916102,NaN,81.581284,NaN,51.287521,2.500392,-0.501950,NaN,-0.558764,163.171646,NaN
35995,15/9-17,473.645998,19.224571,NaN,1.829696,1.924672,NaN,84.375999,NaN,79.293236,2.427329,-1.385147,NaN,-0.512869,162.569824,NaN
35996,15/9-17,473.797998,19.305397,NaN,1.873184,1.928222,NaN,86.059189,NaN,85.377029,2.295868,0.417226,NaN,-0.498815,162.236664,NaN


In [68]:
pd.set_option('display.max_rows', 20)
well_datacount=well.groupby(['WELL'], as_index = False)
well_datacount['DRHO'].quantile(0.005)

,WELL,DRHO
0,15/9-13,-0.026582
1,15/9-15,-0.050337
2,15/9-17,-0.539703
3,16/1-2,-0.200808
4,16/1-6 A,NaN
...,...,...
93,35/9-5,-0.019345
94,35/9-6 S,-0.088358
95,36/7-3,2.241756
96,7/1-1,-0.030776


In [69]:
#Filter RDEP (should be in range 0-2000)
well.drop(well[(well['DRHO'] < -0.4)].index, inplace=True)

C:\Users\V330\AppData\Local\Temp\ipykernel_19304\1530311284.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  well.drop(well[(well['DRHO'] < -0.4)].index, inplace=True)


In [70]:
well.describe()

,DEPTH,CALI,RSHA,RMED,RDEP,RXO,GR,NPHI,PEF,RHOB,SP,BS,DRHO,DTC,DTS
count,1.131070e+06,1.044575e+06,624115.000000,1.092661e+06,1.120069e+06,291976.000000,1.131070e+06,755497.000000,634746.000000,971357.000000,826826.000000,664463.000000,948619.000000,1.052063e+06,174555.000000
mean,2.182755e+03,1.313339e+01,10.387129,3.711397e+00,1.093582e+01,9.753489,7.075523e+01,0.332806,6.215619,2.286507,61.675871,11.887369,0.025119,1.133010e+02,204.669581
std,9.921612e+02,3.795552e+00,98.307498,2.231048e+01,1.158379e+02,95.442252,3.447713e+01,0.130039,10.918190,0.251109,75.607409,3.326543,0.196604,2.991358e+01,71.073957
min,1.360860e+02,2.344000e+00,0.000100,3.990377e-02,3.170056e-02,0.002298,1.092843e-01,-0.035822,0.099718,0.720971,-999.000000,6.000000,-0.399639,7.415132e+00,69.163177
25%,1.421479e+03,9.370763e+00,0.855425,9.161193e-01,9.117858e-01,0.873880,4.730957e+01,0.243026,3.398525,2.095069,33.939554,8.500000,-0.009347,8.786842e+01,155.945190
50%,2.077678e+03,1.253910e+01,1.405115,1.452493e+00,1.437505e+00,1.578709,6.816999e+01,0.327783,4.283869,2.323007,55.880093,12.250001,0.001645,1.093858e+02,188.210999
75%,2.866095e+03,1.633590e+01,3.130927,2.680056e+00,2.559468e+00,4.052797,8.861448e+01,0.423322,5.896415,2.488884,84.843786,12.250001,0.021569,1.407251e+02,224.655365
max,5.436632e+03,2.827900e+01,2193.904541,1.988616e+03,1.999887e+03,2000.000000,1.076964e+03,0.999570,383.129974,3.457820,526.547302,26.000002,2.836938,3.204789e+02,676.578125


### PEF

In [91]:
well['PEF'].quantile(0.98)

27.014279556699943

In [74]:
pd.set_option('display.max_rows', None)
well_datacount=well.groupby(['WELL'], as_index = False)
well_datacount['PEF'].quantile(0.99)

,WELL,PEF
0,15/9-13,10.221097
1,15/9-15,13.304902
2,15/9-17,10.569006
3,16/1-2,NaN
4,16/1-6 A,8.279543
5,16/10-1,18.822380
6,16/10-2,26.043201
7,16/10-3,8.843380
8,16/10-5,9.385436
9,16/11-1 ST3,NaN


In [103]:
pefoutlier = well[(well['PEF'] > 20) & (well['PEF'] < 50)]
pefoutlier['WELL'].unique()
#drhooutlier[drhooutlier['WELL']=='15/9-13']

array(['15/9-15', '15/9-17', '16/10-1', '16/10-2', '16/2-6', '16/7-5',
       '25/11-19 S', '25/2-13 T4', '25/2-14', '25/3-1', '25/5-1',
       '25/7-2', '25/8-5 S', '26/4-1', '29/6-1', '30/3-3', '30/6-5',
       '31/2-7', '31/2-8', '31/2-9', '31/3-2', '31/4-5', '33/9-17',
       '34/10-21', '34/10-35', '34/11-2 S', '34/12-1', '34/2-4',
       '34/4-10 R', '34/7-21', '34/8-1', '35/11-1', '35/11-6', '35/12-1',
       '35/4-1', '35/8-4'], dtype=object)

16/10-2
25/11-19 S
25/7-2
26/4-1	
29/6-1	
30/3-3	
30/6-5
31/2-7
31/2-9
31/3-2
33/9-17
34/10-21
34/10-35
34/11-2 S
34/2-4
34/8-1
35/11-6

In [104]:
pefoutlier[pefoutlier['WELL']=='16/10-2']

,WELL,DEPTH,CALI,RSHA,RMED,RDEP,RXO,GR,NPHI,PEF,RHOB,SP,BS,DRHO,DTC,DTS
76450,16/10-2,2779.577202,15.141150,NaN,1.475515,1.265378,NaN,37.447510,0.450693,23.687037,2.083333,95.075165,12.250001,-0.012656,86.810219,NaN
76451,16/10-2,2779.729202,16.762106,NaN,1.130740,1.204020,NaN,37.722324,0.479017,34.687775,1.877014,95.508911,12.250001,-0.021177,96.371147,NaN
76452,16/10-2,2779.881202,17.212368,NaN,0.923620,1.177672,NaN,39.660786,0.496600,39.776432,1.749578,96.132431,12.250001,-0.020954,104.217117,NaN
76453,16/10-2,2780.033202,17.157625,NaN,0.840949,1.174214,NaN,43.087860,0.477400,33.456791,1.716013,96.496811,12.250001,-0.016203,107.607590,NaN
76454,16/10-2,2780.185202,17.073273,NaN,0.855567,1.186890,NaN,44.904297,0.418132,26.152924,1.793320,96.404198,12.250001,-0.009598,110.577957,NaN
76494,16/10-2,2786.265202,14.384702,NaN,1.551053,1.436763,NaN,26.604170,0.401380,21.863434,2.117249,96.845055,12.250001,0.001359,85.868271,NaN
76495,16/10-2,2786.417202,15.694042,NaN,1.538936,1.469825,NaN,25.964117,0.392473,32.667217,1.938847,96.805267,12.250001,-0.015975,85.987694,NaN
76496,16/10-2,2786.569202,16.803932,NaN,1.491153,1.505824,NaN,25.973192,0.338990,42.793842,1.856138,96.627838,12.250001,-0.048819,91.619621,NaN
76497,16/10-2,2786.721202,17.003073,NaN,1.500779,1.516049,NaN,25.243750,0.285283,23.883194,1.943329,96.431664,12.250001,-0.051485,96.266861,NaN
76498,16/10-2,2786.873202,15.374676,NaN,1.685398,1.519526,NaN,24.735949,0.296170,26.072737,2.056001,96.253647,12.250001,-0.024817,93.836472,NaN


In [4]:
#well_datacount.get_group('25/8-5 S')

well.dtypes.value_counts()

float64    15
object      1
Name: count, dtype: int64

In [6]:
well.isna().sum()

WELL          0
DEPTH         0
CALI      87877
RSHA     539861
RMED      38993
RDEP      11015
RXO      843084
GR            0
NPHI     405102
PEF      498819
RHOB     161269
SP       306264
BS       487854
DRHO     182654
DTC       80863
DTS      995898
dtype: int64

In [10]:
well[well.DTS.isnull()]

,WELL,DEPTH,CALI,RSHA,RMED,RDEP,RXO,GR,NPHI,PEF,RHOB,SP,BS,DRHO,DTC,DTS
0,15/9-13,494.5280,19.480835,NaN,1.611410,1.798681,NaN,80.200851,NaN,20.915468,1.884186,24.612379,NaN,-0.574928,161.131180,NaN
1,15/9-13,494.6800,19.468800,NaN,1.618070,1.795641,NaN,79.262886,NaN,19.383013,1.889794,23.895531,NaN,-0.570188,160.603470,NaN
2,15/9-13,494.8320,19.468800,NaN,1.626459,1.800733,NaN,74.821999,NaN,22.591518,1.896523,23.916357,NaN,-0.574245,160.173615,NaN
3,15/9-13,494.9840,19.459282,NaN,1.621594,1.801517,NaN,72.878922,NaN,32.191910,1.891913,23.793688,NaN,-0.586315,160.149429,NaN
4,15/9-13,495.1360,19.453100,NaN,1.602679,1.795299,NaN,71.729141,NaN,38.495632,1.880034,24.104078,NaN,-0.597914,160.128342,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170506,7/1-2 S,3169.3124,8.423170,NaN,NaN,NaN,NaN,77.654900,NaN,4.586425,2.527984,NaN,8.5,-0.001763,NaN,NaN
1170507,7/1-2 S,3169.4644,8.379244,NaN,NaN,NaN,NaN,75.363937,NaN,7.019858,2.537613,NaN,8.5,-0.007600,NaN,NaN
1170508,7/1-2 S,3169.6164,8.350248,NaN,NaN,NaN,NaN,66.452843,NaN,9.049782,2.491860,NaN,8.5,-0.018297,NaN,NaN
1170509,7/1-2 S,3169.7684,8.313779,NaN,NaN,NaN,NaN,55.784817,NaN,8.903917,2.447539,NaN,8.5,-0.011438,NaN,NaN


In [53]:
well_datacount.apply(lambda x:x['DRH'].quantile(0.99)>200)

,WELL,None
0,15/9-13,False
1,15/9-15,False
2,15/9-17,False
3,16/1-2,True
4,16/1-6 A,False
5,16/10-1,False
6,16/10-2,True
7,16/10-3,False
8,16/10-5,True
9,16/11-1 ST3,False


In [ ]:
well_datacount['DRHO'].any()

In [ ]:
Moving average per 30 cm
Check
Rock quality log -- continuous
R

In [48]:
well[well['WELL'].isin(['33/9-17', '35/11-11', '31/2-19 S','35/12-1', '25/2-7', '25/4-5', '15/9-5', '16/4-1'])]

,WELL,DEPTH,CALI,RSHA,RMED,RDEP,RXO,GR,NPHI,PEF,RHOB,SP,BS,DRHO,DTC,DTS
